<a href="https://colab.research.google.com/github/auramolina/Analitica-en-recursos-humanos/blob/main/giberto1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <font color='0C2054'> **Librerías** </font>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
import seaborn as sns

import warnings


warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score
!pip install kneed
from kneed import KneeLocator

from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA, FactorAnalysis

!pip install factor_analyzer
from factor_analyzer import FactorAnalyzer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for factor_analyzer: filename=factor_analyzer-0.5.1-py2.py3-none-any.whl size=42565 sha256=822addd0559116a3eba62acfa059e785599d23fb282b03be70274eb1a7736f0a
  Stored in directory: /root/.cache/pip/wheels/24/59/82/6493618e30ed1cb7a013b9e1b0c9e17de80b04dfcef4ba8a4d
Successfully built factor_analyzer


## <font color='0c2054'> **Importación de datos** </font>

In [ ]:
# Lectura de los datos
data_general = pd.read_csv("https://raw.githubusercontent.com/auramolina/Analitica-en-recursos-humanos/main/general_data.csv", delimiter=';')
manager_survey = pd.read_csv("https://raw.githubusercontent.com/auramolina/Analitica-en-recursos-humanos/main/manager_survey.csv", delimiter=';')
retirement_info = pd.read_csv("https://raw.githubusercontent.com/auramolina/Analitica-en-recursos-humanos/main/retirement_info.csv", delimiter=',')
employee_survey = pd.read_csv("https://raw.githubusercontent.com/auramolina/Analitica-en-recursos-humanos/main/employee_survey_data.csv", delimiter=';')

## <font color='0c2054'> **Base de datos 1** </font>
data_general




### <font color='0C2054'> **Selección de características** </font>

In [ ]:
data_general.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      8820 non-null   int64  
 1   BusinessTravel           8820 non-null   object 
 2   Department               8820 non-null   object 
 3   DistanceFromHome         8820 non-null   int64  
 4   Education                8820 non-null   int64  
 5   EducationField           8820 non-null   object 
 6   EmployeeCount            8820 non-null   int64  
 7   EmployeeID               8820 non-null   int64  
 8   Gender                   8820 non-null   object 
 9   JobLevel                 8820 non-null   int64  
 10  JobRole                  8820 non-null   object 
 11  MaritalStatus            8820 non-null   object 
 12  MonthlyIncome            8820 non-null   int64  
 13  NumCompaniesWorked       8782 non-null   float64
 14  Over18                  

In [ ]:
data_general.sort_values(by=['EmployeeID'],ascending=1)

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,...,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,InfoDate
0,51,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,1,...,Y,11,8,0,1.0,6,1,0,0,31/12/2015
4410,51,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,1,...,Y,11,8,0,1.0,6,1,0,0,31/12/2016
1,31,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,1,...,Y,23,8,1,6.0,3,5,1,4,31/12/2015
2,32,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,4,...,Y,15,8,3,5.0,2,5,0,3,31/12/2015
4412,32,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,4,...,Y,15,8,3,5.0,2,5,0,3,31/12/2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8798,33,Travel_Rarely,Sales,1,3,Life Sciences,1,8799,Male,2,...,Y,11,8,0,13.0,2,9,1,7,31/12/2016
8801,32,Travel_Rarely,Sales,23,1,Life Sciences,1,8802,Male,3,...,Y,11,8,0,4.0,2,3,1,2,31/12/2016
4391,32,Travel_Rarely,Sales,23,1,Life Sciences,1,8802,Male,3,...,Y,11,8,0,4.0,2,3,1,2,31/12/2015
8812,37,Travel_Frequently,Sales,2,3,Marketing,1,8813,Male,1,...,Y,11,8,1,17.0,2,1,0,0,31/12/2016


In [ ]:
for column in data_general.columns:
    print(f"Frecuencia en la columna: {column}")
    print(data_general[column].value_counts())
    print("\n")

Frecuencia en la columna: Age
Age
35    468
34    462
31    414
36    414
29    408
32    366
30    360
38    348
33    348
40    342
37    300
28    288
27    288
42    276
39    252
45    246
41    240
26    234
46    198
44    198
43    192
50    180
24    156
25    156
49    144
47    144
55    132
48    114
51    114
53    114
52    108
54    108
22     96
58     84
23     84
56     84
21     78
20     66
59     60
19     54
18     48
60     30
57     24
Name: count, dtype: int64


Frecuencia en la columna: BusinessTravel
BusinessTravel
Travel_Rarely        6258
Travel_Frequently    1662
Non-Travel            900
Name: count, dtype: int64


Frecuencia en la columna: Department
Department
Research & Development    5766
Sales                     2676
Human Resources            378
Name: count, dtype: int64


Frecuencia en la columna: DistanceFromHome
DistanceFromHome
2     1266
1     1248
10     516
9      510
3      504
7      504
8      480
5      390
4      384
6      354
16     

En la búsqueda de variables que no aportan información útil, hemos analizado la frecuencia de los datos y hemos identificado que las variables EmployeeCount, Over18 y StandardHours contienen valores que se repiten en todos los registros. Al examinar estos datos, observamos que:

*Over18: El valor en esta columna siempre indica que la persona es mayor de 18 años, lo cual es un requisito para trabajar.

*StandardHours: El valor en esta columna siempre es 8 horas, lo cual es la jornada laboral estándar.

*EmployeeCount: El valor en esta columna es siempre 1, lo que representa el conteo de empleados para cada registro.

Dado que estos valores son constantes y no aportan información adicional sobre la variabilidad de los datos, hemos concluido que estas variables pueden ser descartadas, ya que no tienen utilidad para el análisis.

In [ ]:
# Eliminación de variables no significativas
data_general.drop(['EmployeeCount','Over18','StandardHours'], axis=1, inplace=True)

# Tamaño del dataset
data_general.shape

(8820, 21)

In [ ]:
# Información general del dataset
data_general.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      8820 non-null   int64  
 1   BusinessTravel           8820 non-null   object 
 2   Department               8820 non-null   object 
 3   DistanceFromHome         8820 non-null   int64  
 4   Education                8820 non-null   int64  
 5   EducationField           8820 non-null   object 
 6   EmployeeID               8820 non-null   int64  
 7   Gender                   8820 non-null   object 
 8   JobLevel                 8820 non-null   int64  
 9   JobRole                  8820 non-null   object 
 10  MaritalStatus            8820 non-null   object 
 11  MonthlyIncome            8820 non-null   int64  
 12  NumCompaniesWorked       8782 non-null   float64
 13  PercentSalaryHike        8820 non-null   int64  
 14  StockOptionLevel        

In [ ]:
# Cambiar el tipo de dato de la variable fecha a formato fecha
data_general['InfoDate'] = pd.to_datetime(data_general['InfoDate'], format='%d/%m/%Y')
data_general['InfoDate']

,InfoDate
0,2015-12-31
1,2015-12-31
2,2015-12-31
3,2015-12-31
4,2015-12-31
...,...
8815,2016-12-31
8816,2016-12-31
8817,2016-12-31
8818,2016-12-31


In [ ]:
#observar los nulos de la base de datos
data_general.isnull().sum()

,0
Age,0
BusinessTravel,0
Department,0
DistanceFromHome,0
Education,0
EducationField,0
EmployeeID,0
Gender,0
JobLevel,0
JobRole,0


se observa que se cuentan con datos nulos, no se toca preguntarle al profesor que se hace con ellos o que ?

In [ ]:
data_general[data_general['NumCompaniesWorked'].isnull()]

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeID,Gender,JobLevel,JobRole,...,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,InfoDate
115,27,Travel_Rarely,Sales,2,3,Life Sciences,116,Male,1,Sales Executive,...,23670,NaN,12,1,5.0,2,5,0,4,2015-12-31
210,52,Travel_Rarely,Sales,2,1,Medical,211,Male,1,Laboratory Technician,...,99800,NaN,11,1,18.0,2,10,6,9,2015-12-31
343,40,Travel_Frequently,Sales,9,3,Marketing,344,Male,3,Manufacturing Director,...,22380,NaN,14,0,10.0,3,1,0,0,2015-12-31
476,22,Travel_Rarely,Research & Development,8,3,Other,477,Male,2,Sales Representative,...,49070,NaN,21,1,4.0,2,4,1,1,2015-12-31
647,29,Travel_Rarely,Research & Development,3,3,Technical Degree,648,Female,2,Healthcare Representative,...,48050,NaN,16,1,9.0,2,7,1,7,2015-12-31
799,29,Travel_Frequently,Research & Development,28,3,Medical,800,Female,5,Research Scientist,...,44340,NaN,17,3,7.0,1,5,0,0,2015-12-31
932,32,Travel_Rarely,Research & Development,5,4,Technical Degree,933,Male,5,Research Scientist,...,50030,NaN,13,0,10.0,5,9,1,6,2015-12-31
1103,26,Travel_Frequently,Sales,16,2,Life Sciences,1104,Male,2,Healthcare Representative,...,83800,NaN,14,0,8.0,3,7,7,7,2015-12-31
1312,37,Travel_Rarely,Human Resources,9,3,Human Resources,1313,Male,2,Healthcare Representative,...,54840,NaN,17,0,7.0,4,3,0,2,2015-12-31
1521,31,Travel_Rarely,Research & Development,20,1,Technical Degree,1522,Female,3,Research Scientist,...,99070,NaN,13,1,6.0,3,5,0,3,2015-12-31


In [ ]:
data_general[data_general['TotalWorkingYears'].isnull()]

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeID,Gender,JobLevel,JobRole,...,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,InfoDate
23,42,Travel_Rarely,Research & Development,4,4,Life Sciences,24,Male,1,Manufacturing Director,...,89260,1.0,14,0,NaN,4,20,11,6,2015-12-31
137,43,Travel_Frequently,Sales,14,3,Life Sciences,138,Female,1,Manufacturing Director,...,37480,7.0,14,2,NaN,3,3,1,2,2015-12-31
308,47,Travel_Frequently,Research & Development,4,3,Life Sciences,309,Male,1,Research Director,...,27740,4.0,13,1,NaN,2,5,1,3,2015-12-31
574,37,Travel_Rarely,Research & Development,23,3,Life Sciences,575,Male,3,Manufacturing Director,...,166590,7.0,16,1,NaN,2,6,1,3,2015-12-31
1517,36,Non-Travel,Research & Development,5,4,Life Sciences,1518,Male,1,Sales Executive,...,85780,7.0,21,0,NaN,4,9,0,8,2015-12-31
2367,39,Travel_Rarely,Sales,2,4,Life Sciences,2368,Male,1,Laboratory Technician,...,72640,0.0,17,0,NaN,5,10,0,7,2015-12-31
3120,35,Travel_Rarely,Research & Development,12,2,Medical,3121,Female,2,Sales Representative,...,28090,4.0,23,1,NaN,2,8,0,0,2015-12-31
3818,28,Travel_Rarely,Human Resources,1,3,Medical,3819,Female,2,Laboratory Technician,...,24130,1.0,16,1,NaN,2,1,0,0,2015-12-31
4409,40,Travel_Rarely,Research & Development,28,3,Medical,4410,Male,2,Laboratory Technician,...,54680,0.0,12,0,NaN,6,21,3,9,2015-12-31
4433,42,Travel_Rarely,Research & Development,4,4,Life Sciences,24,Male,1,Manufacturing Director,...,89260,1.0,14,0,NaN,4,20,11,6,2016-12-31


## <font color='0c2054'> **base de datos 2** </font>

### <font color='0C2054'> **Selección de características** </font>

In [ ]:
manager_survey.columns

Index(['EmployeeID', 'JobInvolvement', 'PerformanceRating', 'SurveyDate'], dtype='object')

In [ ]:
#Lectura de datos
manager_survey.sort_values(by=['EmployeeID'], ascending=1)

,EmployeeID,JobInvolvement,PerformanceRating,SurveyDate
0,1,3,3,31/12/2015
4410,1,3,3,31/12/2016
1,2,2,4,31/12/2015
2,3,3,3,31/12/2015
4412,3,3,3,31/12/2016
...,...,...,...,...
8798,8799,3,3,31/12/2016
8801,8802,3,3,31/12/2016
4391,8802,3,3,31/12/2015
8812,8813,3,3,31/12/2016


In [ ]:
# Información general del dataset
manager_survey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   EmployeeID         8820 non-null   int64 
 1   JobInvolvement     8820 non-null   int64 
 2   PerformanceRating  8820 non-null   int64 
 3   SurveyDate         8820 non-null   object
dtypes: int64(3), object(1)
memory usage: 275.8+ KB


In [ ]:
#observar los nulos de la base de datos
manager_survey.isnull().sum()

,0
EmployeeID,0
JobInvolvement,0
PerformanceRating,0
SurveyDate,0


## <font color='0c2054'> **base de datos 3** </font>

## <font color='0c2054'> **SELECCIÓN DE CARACTERÍSTICAS** </font>

In [ ]:
print(retirement_info.columns)

Index(['Unnamed: 0.1', 'Unnamed: 0', 'EmployeeID', 'Attrition',
       'retirementDate', 'retirementType', 'resignationReason'],
      dtype='object')


In [ ]:
#Lectura de datos
retirement_info.sort_values(by=['EmployeeID'],ascending=1)

,Unnamed: 0.1,Unnamed: 0,EmployeeID,Attrition,retirementDate,retirementType,resignationReason
0,0,0,2,Yes,2016-10-15,Resignation,Others
1,1,1,7,Yes,2016-07-20,Resignation,Stress
2,2,2,14,Yes,2016-11-08,Resignation,Others
3,3,3,29,Yes,2016-10-15,Resignation,Others
4,4,4,31,Yes,2016-02-08,Resignation,Salary
...,...,...,...,...,...,...,...
706,706,706,4382,Yes,2015-04-18,Resignation,Salary
707,707,707,4387,Yes,2015-09-16,Resignation,Others
708,708,708,4389,Yes,2015-05-24,Resignation,Others
709,709,709,4392,Yes,2015-02-26,Fired,NaN


In [ ]:
# Información general del dataset
retirement_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 711 entries, 0 to 710
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0.1       711 non-null    int64 
 1   Unnamed: 0         711 non-null    int64 
 2   EmployeeID         711 non-null    int64 
 3   Attrition          711 non-null    object
 4   retirementDate     711 non-null    object
 5   retirementType     711 non-null    object
 6   resignationReason  641 non-null    object
dtypes: int64(3), object(4)
memory usage: 39.0+ KB


In [ ]:
# Eliminación de variables no significativas
retirement_info.drop(['Unnamed: 0.1','Unnamed: 0'], axis=1, inplace=True)

# Tamaño del dataset
retirement_info.shape

(711, 5)

se observa que las variables 'Unnamed: 0.1' y 'Unnamed: 0' son duplicados de la variable 'EmployeeID' por lo que se decide eliminarlas




In [ ]:
#observar los nulos de la base de datos
retirement_info.isnull().sum()

,0
EmployeeID,0
Attrition,0
retirementDate,0
retirementType,0
resignationReason,70


se le pregunta al profesor que se hace con esos datos

## <font color='0c2054'> **base de datos 4** </font>

## <font color='0c2054'> **SELECCIÓN DE CARACTERÍSTICAS** </font>

In [ ]:
print(employee_survey.columns)

Index(['EmployeeID', 'EnvironmentSatisfaction', 'JobSatisfaction',
       'WorkLifeBalance', 'DateSurvey'],
      dtype='object')


In [ ]:
#Lectura de datos
employee_survey.sort_values(by=['EmployeeID'],ascending=1)

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,DateSurvey
0,1,3.0,4.0,2.0,31/12/2015
4410,1,3.0,4.0,2.0,31/12/2016
1,2,3.0,2.0,4.0,31/12/2015
2,3,2.0,2.0,1.0,31/12/2015
4412,3,2.0,2.0,1.0,31/12/2016
...,...,...,...,...,...
8798,8799,1.0,1.0,4.0,31/12/2016
8801,8802,4.0,3.0,1.0,31/12/2016
4391,8802,4.0,3.0,1.0,31/12/2015
8812,8813,1.0,3.0,3.0,31/12/2016


In [ ]:
# Información general del dataset
employee_survey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               8820 non-null   int64  
 1   EnvironmentSatisfaction  8770 non-null   float64
 2   JobSatisfaction          8780 non-null   float64
 3   WorkLifeBalance          8744 non-null   float64
 4   DateSurvey               8820 non-null   object 
dtypes: float64(3), int64(1), object(1)
memory usage: 344.7+ KB


In [ ]:
#observar los nulos de la base de datos
employee_survey.isnull().sum()

,0
EmployeeID,0
EnvironmentSatisfaction,50
JobSatisfaction,40
WorkLifeBalance,76
DateSurvey,0


## <font color='0c2054'> **base de datos completa** </font>

In [ ]:
# Concatenar por filas
Dat_completos = data_general.merge(manager_survey, on='EmployeeID').merge(retirement_info, on='EmployeeID').merge(employee_survey,on='EmployeeID')

# Mostrar el DataFrame concatenado
Dat_completos.sort_values(by=['EmployeeID'],ascending=1)

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,...,PerformanceRating,SurveyDate,Attrition,retirementDate,retirementType,resignationReason,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,DateSurvey
0,31,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,1,...,4,31/12/2015,Yes,2016-10-15,Resignation,Others,3.0,2.0,4.0,31/12/2015
1,28,Travel_Rarely,Research & Development,11,2,Medical,1,7,Male,2,...,4,31/12/2015,Yes,2016-07-20,Resignation,Stress,1.0,3.0,1.0,31/12/2015
2,47,Non-Travel,Research & Development,1,1,Medical,1,14,Male,1,...,3,31/12/2015,Yes,2016-11-08,Resignation,Others,1.0,2.0,2.0,31/12/2015
3,44,Travel_Frequently,Research & Development,1,2,Medical,1,29,Male,2,...,3,31/12/2015,Yes,2016-10-15,Resignation,Others,4.0,3.0,1.0,31/12/2015
4,26,Travel_Rarely,Research & Development,4,3,Medical,1,31,Male,3,...,3,31/12/2015,Yes,2016-02-08,Resignation,Salary,1.0,2.0,3.0,31/12/2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,34,Travel_Frequently,Research & Development,2,1,Life Sciences,1,3973,Male,1,...,4,31/12/2015,Yes,2016-11-11,Resignation,Others,4.0,1.0,2.0,31/12/2015
646,37,Travel_Rarely,Research & Development,24,3,Life Sciences,1,3974,Male,1,...,3,31/12/2015,Yes,2016-04-14,Resignation,Others,4.0,1.0,1.0,31/12/2015
647,26,Travel_Rarely,Human Resources,2,1,Technical Degree,1,3980,Female,3,...,3,31/12/2015,Yes,2016-07-13,Resignation,Others,4.0,2.0,3.0,31/12/2015
648,46,Travel_Rarely,Research & Development,10,3,Medical,1,3981,Male,1,...,3,31/12/2015,Yes,2016-02-18,Resignation,Others,4.0,2.0,4.0,31/12/2015


In [ ]:
#observar los nulos de la base de datos
Dat_completos.isnull().sum()

,0
Age,18351
BusinessTravel,18351
Department,18351
DistanceFromHome,18351
Education,18351
EducationField,18351
EmployeeCount,18351
EmployeeID,0
Gender,18351
JobLevel,18351
